## Importo librerías

In [35]:
import os
from PIL import Image
import numpy as np
import pandas as pd

## Cargo datos de test

In [36]:
# Directorio donde están tus fotos
directorio_fotos = './data/images/test'

# Lista para almacenar los datos
datos = []

# Recorrer el directorio con las fotos
for nombre_foto in os.listdir(directorio_fotos):
    if nombre_foto.endswith('.jpg'):  
        
        with Image.open(os.path.join(directorio_fotos, nombre_foto)) as img:
            # Convertir la imagen a escala de grises y luego a un array de numpy
            array_pixeles = np.array(img.convert('L'))
            # Aplanar el array para tener una sola fila de píxeles
            array_pixeles = array_pixeles.flatten()
            # Añadir el nombre de la foto y el array de píxeles a la lista de datos
            datos.append({'ID': nombre_foto, 'Pixeles': array_pixeles})

# Crear el dataframe
df = pd.DataFrame(datos)

# Mostrar el dataframe
print(df)


             ID                                            Pixeles
0     10004.jpg  [1, 1, 1, 2, 7, 15, 16, 9, 11, 14, 13, 14, 31,...
1     10019.jpg  [159, 157, 158, 166, 170, 170, 172, 171, 166, ...
2     10023.jpg  [195, 198, 193, 190, 140, 70, 54, 88, 169, 212...
3     10029.jpg  [41, 48, 57, 66, 68, 67, 75, 85, 94, 105, 109,...
4      1003.jpg  [44, 33, 21, 23, 29, 29, 34, 37, 40, 40, 45, 4...
...         ...                                                ...
7061   9977.jpg  [178, 139, 130, 131, 159, 188, 183, 239, 249, ...
7062   9979.jpg  [243, 243, 238, 196, 185, 176, 168, 162, 157, ...
7063   9982.jpg  [203, 229, 234, 238, 233, 232, 238, 235, 229, ...
7064   9984.jpg  [57, 56, 57, 63, 61, 56, 58, 58, 61, 72, 69, 6...
7065   9993.jpg  [107, 111, 106, 119, 105, 13, 6, 10, 9, 13, 10...

[7066 rows x 2 columns]


In [37]:
df['Pixeles'][0].size

2304

In [54]:
X_test = np.array(df['Pixeles'].to_list())
X_test = X_test.reshape(-1,48,48)
X_test.shape

(7066, 48, 48)

In [55]:
X_test = X_test/255

In [40]:
df['ID'] = df['ID'].apply(lambda x: x.replace('.jpg','')).astype('int')
df.ID

0       10004
1       10019
2       10023
3       10029
4        1003
        ...  
7061     9977
7062     9979
7063     9982
7064     9984
7065     9993
Name: ID, Length: 7066, dtype: int32

## Cargo datos de train

In [44]:
# Directorio donde están las imágenes
base_dir = './data/images/train'

# Diccionario de etiquetas
dicc_sent = {
    'angry': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3,
    'neutral': 4,
    'sad': 5,
    'surprise': 6
}

# Listas para almacenar imágenes y etiquetas
X = []
y = []

# Iterar sobre las carpetas en el directorio base
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    label = dicc_sent[folder_name]
    
    # Iterar sobre cada archivo de imagen en la carpeta
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        
        # Cargar la imagen en escala de grises usando PIL
        img = Image.open(img_path).convert('L')
        
        # Convertir la imagen a un array de NumPy
        img_array = np.array(img)
        
        # Agregar la imagen y su etiqueta a las listas
        X.append(img_array)
        y.append(label)

# Convertir listas a arrays de NumPy
X = np.array(X)
y = np.array(y)

# Verificar las dimensiones de los arrays
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)


# Normalizar las imágenes
X = X / 255.0

Dimensiones de X: (28821, 48, 48)
Dimensiones de y: (28821,)


## Cargamos el modelo

In [56]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('best_model.keras')
predictions = model.predict(X_test)
predictions

221/221 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step


array([[7.8055668e-01, 8.9818733e-03, 1.2107862e-01, ..., 2.1878291e-02,
        3.6318485e-02, 1.6421525e-02],
       [4.5219162e-03, 3.5706221e-03, 2.4969140e-02, ..., 1.8749794e-02,
        3.0145790e-02, 7.4278428e-03],
       [1.2356385e-01, 4.0236357e-04, 1.3211977e-01, ..., 2.2810379e-01,
        1.1960952e-01, 5.0143078e-03],
       ...,
       [3.3733916e-02, 9.6485246e-04, 2.5416508e-02, ..., 1.1538694e-01,
        7.4044518e-02, 6.0241511e-03],
       [8.2126208e-02, 5.6384620e-03, 1.8081212e-01, ..., 3.3457670e-01,
        1.9792198e-01, 1.3341463e-01],
       [2.2322027e-01, 2.1067874e-04, 2.7795739e-02, ..., 5.8242345e-01,
        1.5003496e-01, 2.6409018e-03]], dtype=float32)

In [58]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes.size

7066

## Archivo submision

In [70]:
sample = pd.DataFrame()
sample['id_img']= df['ID']
sample['label']=predicted_classes
sample

,id_img,label
0,10004,0
1,10019,3
2,10023,3
3,10029,2
4,1003,5
...,...,...
7061,9977,3
7062,9979,3
7063,9982,3
7064,9984,4


In [71]:
dicc_sent_inv = {
    0 : 'angry' ,
    1:'disgust',
    2: 'fear',
    3:'happy',
    4:'neutral',
    5:'sad',
    6:'surprise'
}

sample['label'] = sample['label'].map(dicc_sent_inv)
sample

,id_img,label
0,10004,angry
1,10019,happy
2,10023,happy
3,10029,fear
4,1003,sad
...,...,...
7061,9977,happy
7062,9979,happy
7063,9982,happy
7064,9984,neutral


In [72]:
sample.to_csv('./data/submission_1.csv',index=False)